In [6]:
import re
from dataclasses import dataclass
from typing import List, Union

import pandas
import requests
import tensorflow as tf
from IPython.display import YouTubeVideo

record = "/media/watemerald/Seagate/data/yt8m/video/train0001.tfrecord"

features = []
for example in tf.compat.v1.python_io.tf_record_iterator(record):
    tf_example = tf.train.Example.FromString(example)
    features.append(tf_example)


def expand_vid_id(short_id: Union[bytes, str]) -> str:
    """
    """
    # If the short_id is passed as bytes, that means that is was
    # decoded from a TFRecord directly, in which case it's a UTF-8
    # string
    if isinstance(short_id, bytes):
        short_id = short_id.decode("UTF-8")

    url = f"http://data.yt8m.org/2/j/i/{short_id[:2]}/{short_id}.js"
    val = requests.get(url)

    # The return format looks like i("02ab","tvvJFX90eh0");
    # with the short id on the left and full id on the right
    match = re.match(r"i\(\"(?P<short_id>\w{4})\".\"(?P<full_id>\w+)\"\);", val.text)

    return match.group("full_id")


vocabulary = pandas.read_csv("./video_search/vocabulary.csv")


def label_id_to_name(label: int) -> str:
    """Converts a single label id number to its full Knowledge Graph Name
    """
    return vocabulary.iloc[label]["Name"]


@dataclass
class VideoInfo:
    short_id: str
    long_id: str
    tags: List[str]


def decode_tf_example(e: tf.train.Example) -> VideoInfo:
    short_id = e.features.feature["id"].bytes_list.value[0]
    labels = e.features.feature["labels"].int64_list.value

    long_id = expand_vid_id(short_id)
    tags = list(map(label_id_to_name, labels))
    return VideoInfo(short_id=short_id.decode("UTF-8"), long_id=long_id, tags=tags,)


def display_video(vid: VideoInfo) -> YouTubeVideo:

    return YouTubeVideo(vid.long_id)


print()


In [7]:
f = features[0]

In [8]:
f

features {
  feature {
    key: "id"
    value {
      bytes_list {
        value: "2vab"
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 14
      }
    }
  }
  feature {
    key: "mean_audio"
    value {
      float_list {
        value: 0.4810151159763336
        value: -0.1483372300863266
        value: -0.6934748888015747
        value: 1.2363330125808716
        value: 0.53005450963974
        value: 1.1766775846481323
        value: -1.1011595726013184
        value: -1.2161922454833984
        value: -0.6348494291305542
        value: 0.4173985421657562
        value: 1.2166063785552979
        value: 1.1578223705291748
        value: 1.066794514656067
        value: -1.521916151046753
        value: -0.15031781792640686
        value: 1.3953348398208618
        value: -1.5193017721176147
        value: -1.3475450277328491
        value: 0.8831541538238525
        value: 1.8322529792785645
        value: 0.27107253670692444
        v

In [34]:
a = f.features.feature["mean_rgb"].float_list.value

In [35]:
a

[0.3885611593723297, -1.411874532699585, -0.1713913083076477, -0.7175588607788086, -0.9987234473228455, 0.6637047529220581, 1.2239741086959839, 0.5244296789169312, -0.35780444741249084, 0.6940473914146423, 0.5056536793708801, 0.27939102053642273, -0.7094780802726746, 1.465051531791687, -0.8814725279808044, 0.6699634194374084, -0.5614883303642273, 1.0889770984649658, -0.07426315546035767, 0.09218564629554749, -0.15689338743686676, 0.07340964674949646, 0.4448099136352539, -1.0054574012756348, 1.0237760543823242, 1.2142295837402344, 0.12419198453426361, -0.8138155341148376, 0.15683211386203766, 0.6849367022514343, 0.23724406957626343, -0.11878682672977448, 0.2085651308298111, 0.5966023802757263, 0.25364336371421814, -0.26455825567245483, -0.8909793496131897, 0.2789157032966614, -0.7628747820854187, -0.3531302511692047, 0.676776647567749, 0.5020886063575745, 0.1969984769821167, 0.200405091047287, -0.5245701670646667, 0.19272039830684662, -0.16671711206436157, -0.43774107098579407, 0.113655